In [407]:
import pandas as pd
from collections import Counter
from nltk import WordNetLemmatizer
import re
from nltk.corpus import stopwords

In [408]:
df_triples = pd.read_csv(
    '../../data/processed/reference_model_f/reference_model_f.txt', header=None, sep="\t"
)

df_triples = pd.DataFrame(
    df_triples[0].str.split("|", 4).tolist(),
    columns=["status", "article_id", "e1", "r", "e2"],
)

In [409]:
#df_triples = df_triples.groupby('article_id').head(10).reset_index(drop=True)

In [410]:
def remove_stops(word):
    if word.lower() in set(set(stopwords.words('english'))):
        return ''
    else:
        return word

In [411]:
def extract_entities(text):
    res = []
    for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
        if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
            res.append((' '.join(c[0] for c in chunk.leaves())))
    if len(res) == 0:
        return text
    else:
        return res[0]

In [412]:
lemmatizer = WordNetLemmatizer()
df_triples["l1"] = (
    df_triples["e1"]
        .apply(lambda x: extract_entities(x))
        .apply(lemmatizer.lemmatize)
        .apply(lambda x: x.lower().strip())
        .apply(remove_stops)
        .apply(lambda x: re.sub("[^\s'_A-Za-z]", "", x))
        .apply(lambda x: x.lstrip().rstrip())
)
df_triples["l2"] = (
    df_triples["e2"]
        .apply(lambda x: extract_entities(x))
        .apply(lemmatizer.lemmatize)
        .apply(lambda x: x.lower().strip())
        .apply(remove_stops)
        .apply(lambda x: re.sub("[^\s'_A-Za-z]", "", x))
        .apply(lambda x: x.lstrip().rstrip())
)
df_triples["rel"] = (
    df_triples["r"]
        .apply(lemmatizer.lemmatize)
        .apply(lambda x: x.lower().strip())
        .apply(remove_stops)
        .apply(lambda x: re.sub("[^\s'_A-Za-z]", "", x))
        .apply(lambda x: x.lstrip().rstrip())
)

total_entities = pd.concat([df_triples["l1"], df_triples["l2"]])
c = Counter(total_entities)
un = Counter(el for el in c.elements() if c[el] > 3 and el != '')
unique_entities = pd.Series(list(un.keys()))

total_relations = pd.Series(df_triples["rel"])
rc = Counter(total_relations)
unr = Counter(el for el in rc.elements() if rc[el] > 3 and el != '')
unique_relations = pd.Series(list(unr.keys()))

d = {
    k: v for k, v in zip(unique_entities, [i for i in range(len(unique_entities))])
}
d_sorted = sorted(d.items(), key=lambda kv: kv[1])
r = {
    k: v
    for k, v in zip(
        unique_relations,
        [i for i in range(len(unique_relations))],
    )
}
r_sorted = sorted(r.items(), key=lambda kv: kv[1])

In [413]:
df_fake = df_triples[df_triples['status'] == 'fake'].reset_index()
df_true = df_triples[df_triples['status'] == 'true'].reset_index()

In [418]:
r = Counter(df_fake['rel'])
ents = Counter(pd.concat([df_fake['l1'], df_fake['l2']]))

In [ ]:
len(df_fake.groupby('article_id'))

In [422]:
sum(r.values())

17743